# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import cv2
import json
from PIL import Image
import shutil
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
image_paths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filenames == "train_images":
            image_paths.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

def drawAnnotation(annotaions, img):
    annotaions = json.loads(annotaions.replace("\'", "\""))
    for i in annotaions:
        width, height = i['width'], i['height']
        x, y = i['x'], i['y']
        cv2.rectangle(img, (x,y), (x+width, y+height), (0,255,255), 4)
    
    return img

def show_annotations_and_images(k):
    annotations = k["annotations"]
    print(annotations)
    v_n_f = k['image_id']
    video_number = v_n_f.split("-")[0]
    videoFrame = v_n_f.split("-")[1]
    image_path = "../input/tensorflow-great-barrier-reef/train_images/video_{}/{}.jpg".format(video_number, videoFrame)
    img = cv2.imread(image_path)
    img = drawAnnotation(annotations, img)
    plt.figure(figsize=(20,10))
#     img[:,:,0] = np.zeros([img.shape[0], img.shape[1]])+100
#     plt.imshow(img[...,::-1])
#     plt.imshow(img[:,:,::-1])
    plt.imshow(img)

def ret_full_image_path(d):
    image_path_list = []
    class_name = []
    for i, k in d.iterrows():
        v_n_f = k['image_id']
        video_number = v_n_f.split("-")[0]
        videoFrame = v_n_f.split("-")[1]
        ll = video_number 
        image_path = "../input/tensorflow-great-barrier-reef/train_images/video_{}/{}.jpg".format(video_number, videoFrame)
        image_path_list.append(image_path)
        class_name.append(0)
    return image_path_list, class_name

!pwd

data = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")
print("Total images = ",data.shape)
image_dir = "../input/tensorflow-great-barrier-reef/train_images/video_0"
image_names = []
for img in os.listdir(image_dir):
    image_names.append(img)
    
data = data[data['annotations'] != "[]"]
print("Total images with annotations = ",data.shape)

img_path_list, class_name_list = ret_full_image_path(data)
data = data.assign(image_path = img_path_list)
data = data.assign(Class = class_name_list)

show_annotations_and_images(data.loc[4438])

data

train = data[["image_path","image_id", "annotations", "Class"]]

train.head(50)

for idx, r in train.iterrows():
    annotaitons = r["annotations"]
    annotations = json.loads(annotaitons.replace("\'", "\""))
    print(r["image_path"])
    break

p = "./"
for i in os.listdir(p):
    os.remove(p+i)

def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

!mkdir train

tain_folder_list = []
ex = []
for idx, r in tqdm(train.iterrows()):
    image_id = r["image_id"]
    annotaitons = r["annotations"]
    image_name = image_id.split("-")[1]
    vn = image_id.split("-")[0]
    annotations = json.loads(annotaitons.replace("\'", "\""))
    image_path = r["image_path"]
    destination = "./train/"
    shutil.copyfile(image_path, destination+"{}_{}.jpg".format(vn, image_name))
    
    txt_name = "{}_{}.txt".format(vn, image_name)
    if txt_name in tain_folder_list:
        ex.append(image_path)
    
    with open("./train/{}_{}.txt".format(vn, image_name), "w") as f:
        image_path = r["image_path"]
        img = Image.open(image_path)
        ww= int(img.size[0])
        hh= int(img.size[1])
        for i in annotations:
            x_min = i["x"]
            y_min = i["y"]
            x_max = i["width"] + x_min
            y_max = i["height"] + y_min
            b = (x_min, x_max, y_min, y_max)
            x, y, w, h = convert((ww,hh), b)
            writee = "0 {} {} {} {}".format(x, y, w, h)
            f.write(writee + "\n")
            
        tain_folder_list.append("{}_{}.txt".format(vn, image_name))
        
        

len(os.listdir("./train"))

shutil.rmtree("./fishyolov5")

%cd ..

!git clone https://github.com/rukon-uddin/fishyolov5.git
%cd fishyolov5
%pip install -qr requirements.txt
import torch
from fishyolov5 import utils
display = utils.notebook_init()
!cp best.pt /kaggle/working/best.pt

%cd fishyolov5
%pip install -qr requirements.txt

import torch
from fishyolov5 import utils
display = utils.notebook_init()

!cp best.pt /kaggle/working/best.pt

!tensorboard --logdir runs/train

!python train.py --img 640 --batch 16 --epochs 30 --data data/fish.yaml --weights yolov5s.pt

%cd ..

!tensorboard --logdir runs/train --load_fast=false --bind_all

!cp /kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/60.jpg /kaggle/working/60.jpg

!cp best.pt /kaggle/working/best.pt

%cd weights
%ls

%cd /kaggle/working/fishyolov5

!python detect.py --weights /kaggle/working/best.pt --img 640 --conf 0.1 --source /kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/64.jpg
display.Image(filename='/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/64.jpg', width=600)

from tqdm import tqdm
import os
import time

start = time.time() 
image_files = []
for filename in tqdm(os.listdir(r"/kaggle/working/train")):   # prive the appropriate path
    if filename.endswith(".jpg") or filename.endswith(".JPG") or filename.endswith(".png") or filename.endswith(".PNG"):
        image_files.append("/kaggle/working/train/" + filename) # prive the appropriate path (Note: dont forget to put he backslash '/' at the end)
os.chdir("./")
with open("train.txt", "w") as outfile:
    for image in tqdm(image_files):
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

print(f'\nTime: {time.time() - start}')

import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/kaggle/working/best.pt')  # or yolov5m, yolov5l, yolov5x, custom

# Images

# Results

img = '/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/64.jpg'
results = model(img)
print('\n', results.xyxy[0])

img = plt.imread("/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/64.jpg")
for i in results.xyxy[0].tolist():
    print(map(int, i))
    x1, y1, x2, y2, c, clas = map(int, i)   
    print(x1)
    
    cv2.rectangle(img, (x1,y1), (x2, y2), (0,255,255), 4)

plt.figure(figsize=(20,10))
plt.imshow(img)
    

x1, y1, x2, y2, c, clas = map(int, results.xyxy[0].tolist()[0])

In [ ]:
# if x1 > x2:
#     t=x1
#     x1=x2
#     x2=t
    
# if y1 > y2:
#     t=y1
#     y1=y2
#     y2=t


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import cv2
import json
from PIL import Image
import shutil
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

!git clone https://github.com/rukon-uddin/fishyolov5.git

%cd fishyolov5

%cd fishyolov5
%pip install -qr requirements.txt
import torch
from fishyolov5 import utils
display = utils.notebook_init()

!cp best.pt /kaggle/working/best.pt

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
import torch

# Model
model = torch.hub.load('/kaggle/input/yolov5-lib-ds', 'custom', path='/kaggle/input/weights/best.pt', source='local', force_reload=True)  # or yolov5m, yolov5l, yolov5x, custom
model.conf = 0.1

In [ ]:
import greatbarrierreef

env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()

In [ ]:
idc = 0
submission_dict = {
    'index': [],
    'annotations': [],
}

# env.predict(sample_prediction_df)
for (image, sample_prediction_df) in iter_test:
    results = model(image)
    predictions = []
#     print('\n', results.xyxy[0])
    submission_dict['index'].append(idc)
    for i in results.xyxy[0].tolist():
        x1, y1, x2, y2, c, clas = map(int, i)
        print(x1, y1, x2, y2)
        predictions.append('{:.2f} {} {} {} {}'.format(c, x1, y1, x2, y2))
    
    prediction_str = ' '.join(predictions)
    if len(prediction_str)==0:
        prediction_str = "NaN"
    submission_dict['annotations'].append(prediction_str)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)
    idc+=1
    
    
df= pd.DataFrame(submission_dict)
print("Done!!")

In [ ]:
df

%cd ..

In [ ]:
df.to_csv("submission.csv")